In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from urllib import parse
import time
import pandas as pd
import re
from tqdm import tqdm

C:\Users\sons\AppData\Local\Temp\ipykernel_39588\3160177533.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# TAAS GIS 시스템 로드
driver = webdriver.Chrome()
driver.maximize_window()
driver.get('http://taas.koroad.or.kr/gis/mcm/mcl/initMap.do?menuId=GIS_GMP_STS_RSN')
time.sleep(1)

In [4]:
# 체크박스 체크
elements = driver.find_elements(By.NAME, 'ACDNT_GAE_CODE')[0:4]
for element in elements:
    if not element.is_selected():
        element.click()

selectbox = driver.find_element(By.ID, 'ptsRafSigungu')
select = Select(selectbox)
select.select_by_value("01") # 전체 지역구(서울)

time.sleep(1)


In [5]:
# search
driver.find_element(By.XPATH, '//*[@id="regionAccidentFind"]/div/p/a').send_keys(Keys.ENTER)
time.sleep(1)

In [8]:
# Crawling
coord_ls = []
srs_ls = []

for i in tqdm(range(0,33698)):
    driver.execute_script(f'gis.srh.msh.selectListUi({i}, 11);') # 개별 사고지점
    time.sleep(1) # 로드

    bbox = driver.find_elements(By.CLASS_NAME,'olTileImage')[-1]
    # 디버깅 코드
    if isinstance(bbox, list):
        print("Debugging - src content:")
        for element in bbox:
            print(element)
    else:
        print("Debugging - src content:", bbox)

    src = parse.unquote(bbox.get_attribute('src')) # get src and unquote from 16byte

    # 디버깅 코드
    if isinstance(src, list):
        print("Debugging - src content:")
        for element in src:
            print(element)
    else:
        print("Debugging - src content:", src)

    src_split = src.split('&')
    print("src_split : " + str(src_split))  # Use str() to convert list to string

    if isinstance(src_split, list):
        print("Debugging - src_split:")
        for element in src_split:
            print(element)
    else:
        print("Debugging - src_split:", src_split)


    bbox_text = src_split[-3][5:]
    print("bbox_text : " + bbox_text)


    coords = [float(i) for i in bbox_text.split(',')]
    x, y = (coords[0]+coords[2])/2, (coords[1]+coords[3])/2 # center position
    srs = src_split[-4][4:]
    coord_ls.append([x, y])
    srs_ls.append(srs)

  0%|          | 1/33698 [00:06<58:28:15,  6.25s/it]

Debugging - src content: https://taas.koroad.or.kr/gis36/TAAS/wms?LAYERS=GNRL_GIS_ACDNT_INFO_NON&STYLES=&FORMAT=image/png&TRANSPARENT=TRUE&TILED=false&TILESORIGIN=-30000,-60000&PROJECTION=EPSG:5179&DISPLAYPROJECTION=EPSG:5181&TRANSFORM=EPSG:5181&SERVICE=WMS&VERSION=1.1.1&REQUEST=GetMap&CQL_FILTER=(engn_code='00') and (acdnt_year between '2022' and '2022')  and (legaldong_code ilike '11%') and (( acdnt_gae_code = '01') or ( acdnt_gae_code = '02') or ( acdnt_gae_code = '03') or ( acdnt_gae_code = '04'))&SRS=EPSG:5181&BBOX=176018.02583088,421524.7877365,219410.02583088,481300.7877365&WIDTH=678&HEIGHT=934
src_split : ['https://taas.koroad.or.kr/gis36/TAAS/wms?LAYERS=GNRL_GIS_ACDNT_INFO_NON', 'STYLES=', 'FORMAT=image/png', 'TRANSPARENT=TRUE', 'TILED=false', 'TILESORIGIN=-30000,-60000', 'PROJECTION=EPSG:5179', 'DISPLAYPROJECTION=EPSG:5181', 'TRANSFORM=EPSG:5181', 'SERVICE=WMS', 'VERSION=1.1.1', 'REQUEST=GetMap', "CQL_FILTER=(engn_code='00') and (acdnt_year between '2022' and '2022')  and (le

  0%|          | 2/33698 [00:07<29:51:50,  3.19s/it]

Debugging - src content: https://taas.koroad.or.kr/gis36/TAAS/wms?LAYERS=GNRL_GIS_ACDNT_INFO_NON&STYLES=&FORMAT=image/png&TRANSPARENT=TRUE&TILED=false&TILESORIGIN=-30000,-60000&PROJECTION=EPSG:5179&DISPLAYPROJECTION=EPSG:5181&TRANSFORM=EPSG:5181&SERVICE=WMS&VERSION=1.1.1&REQUEST=GetMap&CQL_FILTER=(engn_code='00') and (acdnt_year between '2022' and '2022')  and (legaldong_code ilike '11%') and (( acdnt_gae_code = '01') or ( acdnt_gae_code = '02') or ( acdnt_gae_code = '03') or ( acdnt_gae_code = '04'))&SRS=EPSG:5181&BBOX=176018.02583088,421524.7877365,219410.02583088,481300.7877365&WIDTH=678&HEIGHT=934
src_split : ['https://taas.koroad.or.kr/gis36/TAAS/wms?LAYERS=GNRL_GIS_ACDNT_INFO_NON', 'STYLES=', 'FORMAT=image/png', 'TRANSPARENT=TRUE', 'TILED=false', 'TILESORIGIN=-30000,-60000', 'PROJECTION=EPSG:5179', 'DISPLAYPROJECTION=EPSG:5181', 'TRANSFORM=EPSG:5181', 'SERVICE=WMS', 'VERSION=1.1.1', 'REQUEST=GetMap', "CQL_FILTER=(engn_code='00') and (acdnt_year between '2022' and '2022')  and (le

  0%|          | 3/33698 [00:08<20:41:04,  2.21s/it]

...

Debugging - src content: https://taas.koroad.or.kr/gis36/TAAS/wms?LAYERS=GNRL_GIS_ACDNT_INFO_NON&STYLES=&FORMAT=image/png&TRANSPARENT=TRUE&TILED=false&TILESORIGIN=-30000,-60000&PROJECTION=EPSG:5179&DISPLAYPROJECTION=EPSG:5181&TRANSFORM=EPSG:5181&SERVICE=WMS&VERSION=1.1.1&REQUEST=GetMap&CQL_FILTER=(engn_code='00') and (acdnt_year between '2022' and '2022')  and (legaldong_code ilike '11%') and (( acdnt_gae_code = '01') or ( acdnt_gae_code = '02') or ( acdnt_gae_code = '03') or ( acdnt_gae_code = '04'))&SRS=EPSG:5181&BBOX=176018.02583088,421524.7877365,219410.02583088,481300.7877365&WIDTH=678&HEIGHT=934
src_split : ['https://taas.koroad.or.kr/gis36/TAAS/wms?LAYERS=GNRL_GIS_ACDNT_INFO_NON', 'STYLES=', 'FORMAT=image/png', 'TRANSPARENT=TRUE', 'TILED=false', 'TILESORIGIN=-30000,-60000', 'PROJECTION=EPSG:5179', 'DISPLAYPROJECTION=EPSG:5181', 'TRANSFORM=EPSG:5181', 'SERVICE=WMS', 'VERSION=1.1.1', 'REQUEST=GetMap', "CQL_FILTER=(engn_code='00') and (acdnt_year between '2022' and '2022')  and (le

  0%|          | 6/33698 [00:11<12:31:17,  1.34s/it]

Debugging - src content: https://taas.koroad.or.kr/gis36/TAAS/wms?LAYERS=GNRL_GIS_ACDNT_INFO_NON&STYLES=&FORMAT=image/png&TRANSPARENT=TRUE&TILED=false&TILESORIGIN=-30000,-60000&PROJECTION=EPSG:5179&DISPLAYPROJECTION=EPSG:5181&TRANSFORM=EPSG:5181&SERVICE=WMS&VERSION=1.1.1&REQUEST=GetMap&CQL_FILTER=(engn_code='00') and (acdnt_year between '2022' and '2022')  and (legaldong_code ilike '11%') and (( acdnt_gae_code = '01') or ( acdnt_gae_code = '02') or ( acdnt_gae_code = '03') or ( acdnt_gae_code = '04'))&SRS=EPSG:5181&BBOX=176018.02583088,421524.7877365,219410.02583088,481300.7877365&WIDTH=678&HEIGHT=934
src_split : ['https://taas.koroad.or.kr/gis36/TAAS/wms?LAYERS=GNRL_GIS_ACDNT_INFO_NON', 'STYLES=', 'FORMAT=image/png', 'TRANSPARENT=TRUE', 'TILED=false', 'TILESORIGIN=-30000,-60000', 'PROJECTION=EPSG:5179', 'DISPLAYPROJECTION=EPSG:5181', 'TRANSFORM=EPSG:5181', 'SERVICE=WMS', 'VERSION=1.1.1', 'REQUEST=GetMap', "CQL_FILTER=(engn_code='00') and (acdnt_year between '2022' and '2022')  and (le

  0%|          | 6/33698 [00:12<19:28:05,  2.08s/it]


KeyboardInterrupt: 

In [18]:
# 크롤링한 좌표 확인
coord_ls

[[186939.12677225, 451476.66760976],
 [211513.66370824, 446214.23884976],
 [200237.80376256, 451179.27616949],
 [201025.35916494, 452260.88737941],
 [205371.16803268, 453182.36723775],
 [194006.55368465, 444374.52952801],
 [202203.57714653, 446286.80941186],
 [209308.10881437, 445587.27146889],
 [198827.39915092, 442682.47239436],
 [203810.47325011, 442470.8416824],
 [190815.80735357, 453692.12691692],
 [195115.12840635, 441144.17378934],
 [204223.96599433, 463275.2488761],
 [199019.67722382, 451301.84613352],
 [201370.61216133, 459002.36416808],
 [191409.83019276, 440556.28623728],
 [204956.08894429, 456736.56494613],
 [189061.20677385, 443331.87313895],
 [195375.1826299, 449060.60187236],
 [194442.4641915, 446305.58594848],
 [195118.07636777, 441154.19328834],
 [214899.45846914, 450838.10364633],
 [204603.30555948, 454790.91792965],
 [206350.1115816, 449693.1897841],
 [208194.18514866, 455457.29318147],
 [196624.44678215, 441947.49394645],
 [195221.84585769, 454001.68879912],
 [19001

In [27]:
# 크롤링데이터 json 파일로 저장
import json

# JSON 파일로 저장
with open('coord_data5.json', 'w') as json_file:
    json.dump(coord_ls, json_file)


In [29]:
# 크롤링한 좌표 데이터 갯수 체크
import json

# 파일 이름
file_name = 'coord_data.json'

# 파일 내용 읽기
with open(file_name, 'r') as file:
    json_data = json.load(file)

# JSON 데이터의 개수 확인
data_count = len(json_data)

# 결과 출력
print(f"{file_name} 파일의 데이터 개수: {data_count}")


coord_data.json 파일의 데이터 개수: 33699


In [30]:
# 크롤링데이터 csv파일로 변환
import pandas as pd

# JSON 파일 읽기
with open('coord_data.json', 'r') as json_file:
    data = json.load(json_file)

# JSON 데이터를 DataFrame으로 변환
df = pd.DataFrame(data)

# CSV 파일로 저장
df.to_csv('서울시전체교통사고좌표값.csv', index=False)

In [39]:
# 크롤링한 좌표값 변환하는작업
import pandas as pd
from pyproj import Proj, Transformer

# CSV 파일 경로
csv_file_path = '서울시전체교통사고좌표값.csv'

# CSV 파일 읽기
df = pd.read_csv(csv_file_path)

# epsg:5181 좌표계 정의
in_proj = Proj(init='epsg:5181')

# epsg:4326 좌표계 정의
out_proj = Proj(init='epsg:4326')

# 좌표 변환 함수 정의
def convert_coordinates(row):
    lon, lat = row['lon(경도)'], row['lat(위도)']
    lon, lat = Transformer.from_proj(in_proj, out_proj).transform(lon, lat)
    return pd.Series({'lon(경도)': lon, 'lat(위도)': lat})

# 'lon(경도)'와 'lat(위도)' 컬럼 생성 및 좌표 변환
df[['lon(경도)', 'lat(위도)']] = df.apply(convert_coordinates, axis=1)

# 결과를 CSV 파일로 저장
df.to_csv('converted_result.csv', index=False)


c:\Python\Python312\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Python\Python312\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [40]:
# 위도경도 순서바꾸기
import pandas as pd

# CSV 파일 경로
csv_file_path = 'converted_result.csv'

# CSV 파일 읽기
df = pd.read_csv(csv_file_path)

# 'lon(경도)'와 'lat(위도)' 컬럼의 위치 바꾸기
df = df[['lat(위도)', 'lon(경도)']]

# 결과를 CSV 파일로 저장
df.to_csv('swapped_result.csv', index=False)


In [41]:
# 사고데이터와 크롤링후 변환한 좌표데이터 붙이기작업

import pandas as pd

# accidentInfoList.csv 파일 읽기
accident_df = pd.read_csv('accidentInfoList.csv', encoding='cp949')

# swapped_result.csv 파일 읽기(좌표데이터)
coord_df = pd.read_csv('swapped_result.csv')

# 두 DataFrame을 수평으로 붙이기
merged_df = pd.concat([accident_df, coord_df], axis=1)

# 결과를 CSV 파일로 저장
merged_df.to_csv('merged_result.csv', index=False)

In [42]:
# 크롤링한 데이터 좌표 검증에 썼던 코드 (안써도됨 작업하는거아니고 확인하는 코드임)
from pyproj import Proj, transform

# 원래 좌표계와 목표 좌표계 정의
original_crs = "EPSG:5181"
target_crs = "EPSG:4326"

# 좌표 변환
# x, y = 200524.55963108, 442588.44509583
x, y = 202751.69687043,460402.27063206

transformer = Proj(init=original_crs), Proj(init=target_crs)
lon, lat = transform(transformer[0], transformer[1], x, y)

print(f"변환된 좌표 (WGS84): 위도 : {lat}, 경도 : {lon}")
print(f"변환된 좌표 (WGS84): {lat}, {lon}")

변환된 좌표 (WGS84): 위도 : 37.64323741802382, 경도 : 127.03117848977485
변환된 좌표 (WGS84): 37.64323741802382, 127.03117848977485


c:\Python\Python312\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Python\Python312\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\kkt82\AppData\Local\Temp\ipykernel_4140\3738534754.py:13: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon, lat = transform(tr